Google Colab上執行 SourceCode@Google Colab: https://colab.research.google.com/drive/1Ck36L5OqFVtlUe5QmTOka_2O5f-ebhhl

In [1]:
# _*_ coding: utf-8 _*_
# _*_ coding: cp950 _*_

# author: Chi-Hsu Chen (css920@gmail.com)

import numpy as np
import cv2
import time
from google.colab.patches import cv2_imshow

from keras import utils
from keras.preprocessing.image import ImageDataGenerator

from keras.models import Model,Sequential,Input
from keras.layers import Dense,Conv2D,Flatten,MaxPooling2D,Convolution2D
from keras.layers import Input,GlobalAveragePooling2D,GlobalMaxPooling2D
from keras.layers import BatchNormalization,Activation,Dropout,Reshape
from keras import layers
from keras import backend as K
from keras.layers import Lambda
from keras.layers import Concatenate
from keras.layers import TimeDistributed

from keras.datasets import cifar10
from sklearn.preprocessing import OneHotEncoder

import keras

Using TensorFlow backend.


In [2]:
input_shape_img = (1024, 1024, 3)
img_input = Input(shape=input_shape_img)

#'''先過一般CNN層提取特徵'''
def nn_base(img_input):
    # Block 1
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv1')(img_input)
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv2')(x)
    # 縮水1/2 1024x1024 -> 512x512
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool')(x)

    # Block 2
    x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv1')(x)
    x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv2')(x)
    # 縮水1/2 512x512 -> 256x256
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool')(x)

    # Block 3
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv1')(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv2')(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv3')(x)
    # 縮水1/2 256x256 -> 128x128
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool')(x)

    # Block 4
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv1')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv2')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv3')(x)
    # 縮水1/2 128x128 -> 64x64
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool')(x)

    # Block 5
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv1')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv2')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv3')(x)

    # 最後返回的x是64x64x512的feature map。
    return x

In [17]:
# RPN (Region Proposal Network)
def rpn(base_layers, num_anchors):

    x = Conv2D(512, kernel_size=(3,3), padding='same', activation='relu', kernel_initializer='normal', name='rpn_conv1')(base_layers)

    # rpn分類和迴歸
    # RPN輸出
    # class => object / not object 以IOU>0.7決定
    # regression => x,y,width,height
    # 所以，以下x_class的Conv2D中num_anchors x 2 (object / not object)
    # x_reg的Conv2D中num_anchors x 4 (x,y,width,height)
    x_class = Conv2D(num_anchors*2, (1, 1), activation='softmax',name='rpn_out_class')(x)
    x_reg = Conv2D(num_anchors *4, (1, 1), activation='linear', name='rpn_out_regress')(x)

    return x_class, x_reg, base_layers

In [18]:
base_layers=nn_base(img_input)
x_class, x_reg, base_layers=rpn(base_layers, 9)

print('Classification支線：',x_class) #'''確認深度是否為18'''
print('BBOX Regression 支線：',x_reg) #'''確認深度是否為36'''
print('CNN Output：',base_layers)

Classification支線： Tensor("rpn_out_class_5/truediv:0", shape=(None, 64, 64, 18), dtype=float32)
BBOX Regression 支線： Tensor("rpn_out_regress_5/BiasAdd:0", shape=(None, 64, 64, 36), dtype=float32)
CNN Output： Tensor("block5_conv3_5/Relu:0", shape=(None, 64, 64, 512), dtype=float32)
